# **Super mario playing RL Agent using Double Q learning**


Importing the OpenAI Gym environment for Super Mario Bros


In [ ]:
!python3 -m pip install gym-super-mario-bros==7.3.0
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

from pyvirtualdisplay import Display

Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.10).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.


In [ ]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

 TRAIN A MARIO-PLAYING RL AGENT


In [ ]:
# Installing the required libraries
import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os, copy
from IPython import display as ipythondisplay
import time, datetime
import pickle
from tqdm import tqdm
import glob
import io
import base64
from IPython.display import HTML

# Installing Gym an OpenAI toolkit for RL
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack
from gym.wrappers import Monitor

# Installing NES Emulator for OpenAI Gym
from nes_py.wrappers import JoypadSpace

# Installing Super Mario environment for OpenAI Gym
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY


 Functions to Display Mario play the game:

In [ ]:
display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

Initialize the Mario enviornment

The mario environment is intialized using the method make(). 



In [ ]:
# Initialize Super Mario environment
env = wrap_env(gym_super_mario_bros.make("SuperMarioBros-1-1-v0"))

Preprocessing the Environment





In [ ]:
class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        done = False
        for i in range(self._skip):
            # Accumulate reward and repeat the same action
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, reward, done, info


class GrayScaleObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape[:2]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def permute_orientation(self, observation):
        # permute [H, W, C] array to [C, H, W] tensor
        observation = np.transpose(observation, (2, 0, 1))
        observation = torch.tensor(observation.copy(), dtype=torch.float)
        return observation

    def observation(self, observation):
        observation = self.permute_orientation(observation)
        transform = T.Grayscale()
        observation = transform(observation)
        return observation


class ResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        if isinstance(shape, int):
            self.shape = (shape, shape)
        else:
            self.shape = tuple(shape)

        obs_shape = self.shape + self.observation_space.shape[2:]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def observation(self, observation):
        transforms = T.Compose(
            [T.Resize(self.shape), T.Normalize(0, 255)]
        )
        observation = transforms(observation).squeeze(0)
        return observation


# Apply Wrappers to environment
env = SkipFrame(env, skip=4)
env = GrayScaleObservation(env)
env = ResizeObservation(env, shape=84)
env = FrameStack(env, num_stack=4)
# Using the RIGHT_ONLY action list for Mario in the environment
env = JoypadSpace(env, RIGHT_ONLY)

In [ ]:
class Mario:
    def __init__(self, state_dim, action_dim, save_dir, pretrained):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.save_dir = save_dir
        self.pretrained = pretrained
        self.count_explore = 0
        self.count_exploit = 0

        self.use_cuda = torch.cuda.is_available()

        # Mario's DNN to predict the most optimal action when doing exploit
        self.net = MarioNet(self.state_dim, self.action_dim).float()
        
        if self.use_cuda:
            self.net = self.net.to(device="cuda")
        if self.pretrained:
          self.exploration_rate = torch.load('exploration_rate.pth')
          self.net.load_state_dict(torch.load("model.pth"))
        else:
          self.exploration_rate = 1

        self.exploration_rate_decay = 0.99
        self.exploration_rate_min = 0.01
        self.curr_step = 0
        self.save_every = 5e5  # no. of experiences between saving Mario Net

    def get_exploration_rate(self):
      return self.exploration_rate

    def act(self, state):
        # EXPLORE
        if np.random.rand() < self.exploration_rate:
            action_idx = np.random.randint(self.action_dim)
        # EXPLOIT
        else:
            state = state.__array__()
            if self.use_cuda:
                state = torch.tensor(state).cuda()
            else:
                state = torch.tensor(state)
            state = state.unsqueeze(0)
            action_values = self.net(state, model="online")
            action_idx = torch.argmax(action_values, axis=1).item()

        # decrease exploration_rate
        self.exploration_rate *= self.exploration_rate_decay
        self.exploration_rate = max(self.exploration_rate_min, self.exploration_rate)

        # increment step
        self.curr_step += 1
        return action_idx


In [ ]:
class Mario(Mario):  # subclassing for continuity
    def __init__(self, state_dim, action_dim, save_dir, pretrained):
        super().__init__(state_dim, action_dim, save_dir, pretrained)
        
        if pretrained:
          self.memory = torch.load('memory.pth')
        else:
          self.memory = deque(maxlen=30000)
        self.batch_size = 32
       

    def cache(self, state, next_state, action, reward, done):
        state = state.__array__()
        next_state = next_state.__array__()
        if self.use_cuda:
          state = torch.tensor(state).cuda()
          next_state = torch.tensor(next_state).cuda()
          action = torch.tensor([action]).cuda()
          reward = torch.tensor([reward]).cuda()
          done = torch.tensor([done]).cuda()
        else:
          state = torch.tensor(state)
          next_state = torch.tensor(next_state)
          action = torch.tensor([action])
          reward = torch.tensor([reward])
          done = torch.tensor([done])

        self.memory.append((state, next_state, action, reward, done))


    def get_memory(self):
      return self.memory


    def recall(self):
        batch = random.sample(self.memory, self.batch_size)
        state, next_state, action, reward, done = map(torch.stack, zip(*batch))
        return state, next_state, action.squeeze(), reward.squeeze(), done.squeeze()






In [ ]:
class MarioNet(nn.Module):

    def __init__(self, input_dim, output_dim):
        super().__init__()
        c, h, w = input_dim
       

        if h != 84:
            raise ValueError(f"Expecting input height: 84, got: {h}")
        if w != 84:
            raise ValueError(f"Expecting input width: 84, got: {w}")

        self.online = nn.Sequential(
            nn.Conv2d(in_channels=c, out_channels=32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )

        self.target = copy.deepcopy(self.online)

        # Q_target parameters are frozen.
        for p in self.target.parameters():
            p.requires_grad = False

    def forward(self, input, model):
        if model == "online":
            return self.online(input)
        elif model == "target":
            return self.target(input)

In [ ]:
class Mario(Mario):
    def __init__(self, state_dim, action_dim, save_dir, pretrained):
        super().__init__(state_dim, action_dim, save_dir, pretrained)
        self.gamma = 0.9
 
    def td_estimate(self, state, action):
        current_Q = self.net(state, model="online")[
            np.arange(0, self.batch_size), action
        ]  # Q_online(s,a)
        return current_Q

    @torch.no_grad()
    def td_target(self, reward, next_state, done):
        next_state_Q = self.net(next_state, model="online")
        best_action = torch.argmax(next_state_Q, axis=1)
        next_Q = self.net(next_state, model="target")[
            np.arange(0, self.batch_size), best_action
        ]
        return (reward + (1 - done.float()) * self.gamma * next_Q).float()

In [ ]:
class Mario(Mario):
    def __init__(self, state_dim, action_dim, save_dir, pretrained):
        super().__init__(state_dim, action_dim, save_dir, pretrained)
        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=0.00025)
        self.loss_fn = torch.nn.SmoothL1Loss()

    def update_Q_online(self, td_estimate, td_target):
        loss = self.loss_fn(td_estimate, td_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

    def sync_Q_target(self):
        self.net.target.load_state_dict(self.net.online.state_dict())

In [ ]:
class Mario(Mario):
    def __init__(self, state_dim, action_dim, save_dir, pretrained):
        super().__init__(state_dim, action_dim, save_dir, pretrained)
        self.burnin = 1e4  # min. experiences before training
        self.learn_every = 3  # no. of experiences between updates to Q_online
        self.sync_every = 1e4  # no. of experiences between Q_target & Q_online sync

    def learn(self):
        if self.curr_step % self.sync_every == 0:
            self.sync_Q_target()

        if self.curr_step < self.burnin:
            return None, None

        if self.curr_step % self.learn_every != 0:
            return None, None

        # Sample from memory
        state, next_state, action, reward, done = self.recall()

        # Get TD Estimate
        td_est = self.td_estimate(state, action)

        # Get TD Target
        td_tgt = self.td_target(reward, next_state, done)

        # Backpropagate loss through Q_online
        loss = self.update_Q_online(td_est, td_tgt)

        return (td_est.mean().item(), loss)

In [ ]:
def run(training_mode, pretrained, num_episodes=1):
  use_cuda = torch.cuda.is_available()
  print(f"Using CUDA: {use_cuda}")
  print()

  save_dir = Path("checkpoints") / datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
  save_dir.mkdir(parents=True)
  mario = Mario(state_dim=(4, 84, 84), action_dim=env.action_space.n, save_dir=save_dir, pretrained=pretrained)

  
  total_rewards =[]
  for e in tqdm(range(num_episodes)):
    state = env.reset()
    total_reward = 0
    STATE = state
    ACTION = None
    REWARD = None
    NEXT_STATE = None
    DONE = None
    # Play the game!
    while True:
          # Renders to the display
          env.render()
          action = mario.act(state)
          # Agent performs action
          next_state, reward, done, info = env.step(action)
          total_reward += reward
          STATE = state
          ACTION = action
          REWARD = reward
          NEXT_STATE = next_state
          DONE = done
          if training_mode:
            # Remember
            mario.cache(state, next_state, action, reward, done)
            # Learn
            q, loss = mario.learn()
          # Update state
          state = next_state
          # Check if end of game
          if done or info["flag_get"]:
            break

    total_rewards.append(total_reward)
    if e != 0 and e % 100 == 0:
      print("Episode {} score = {}, average score = {}".format(e + 1, total_rewards[-1], np.mean(total_rewards)))

  print("Episode {} score = {}, average score = {}".format(e + 1, total_rewards[-1], np.mean(total_rewards)))
  
  if training_mode:
    #saving model weights
    torch.save(mario.net.state_dict(), "model.pth")
    #saving the exploration rate
    torch.save(mario.get_exploration_rate(), "exploration_rate.pth")
    # saving the memory queue
    torch.save(mario.get_memory(), "memory.pth")
  env.close()
  show_video()


In [ ]:
# runing the game and then testing the game
run(training_mode=True, pretrained=False, num_episodes=10)

Using CUDA: True



  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256
 50%|█████     | 5/10 [03:05<02:45, 33.12s/it]/usr/local/lib/python3.7/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256
 80%|████████  | 8/10 [03:06<00:20, 10.43s/it]/usr/local/lib/python3.7/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256
100%|██████████| 10/10 [03:08<00:00, 18.85s/it]


Episode 10 score = 53, average score = 39.1


In [ ]:
run(training_mode=True, pretrained=False, num_episodes=100)

Using CUDA: True



  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256
  1%|          | 1/100 [00:08<14:41,  8.90s/it]/usr/local/lib/python3.7/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256
  8%|▊         | 8/100 [00:17<01:34,  1.03s/it]/usr/local/lib/python3.7/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256
 27%|██▋       | 27/100 [00:25<00:29,  2.49it/s]/usr/local/lib/python3.7/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256
 64%|██████▍   | 64/100 [00:46<00:14,  2.52it/s]/usr/local/lib/python3.7/dist-packages/gym_supe

Episode 100 score = 53, average score = 59.96


In [ ]:
# Training Mario for another 1000 episodes to see if warm_staring Mario makes a difference
run(training_mode=True, pretrained=True, num_episodes=2000)

In [ ]:
# Testing Mario and gathering video of his performance over one episode
run(training_mode=False, pretrained=True, num_episodes=3)

Using CUDA: True



  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256
 33%|███▎      | 1/3 [00:06<00:13,  6.56s/it]/usr/local/lib/python3.7/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256
100%|██████████| 3/3 [00:12<00:00,  4.22s/it]

Episode 3 score = 392, average score = 392.0
